### Runnable
* Runnable은 런타임에 실행될 수 있는 모든 객체를 의미한다.
### Runnable 종류
* invoke : 입력에 대해 체인을 호출한다.
* stream : 응답의 청크를 스트리밍한다.
* batch : 입력 목록에 대한 일괄 처리를 수행한다.

이 외에 비동기 메소드도 존재한다.
* astream : 비동기적으로 응답의 청크를 비동기 스트리밍한다.
* ainvoke : 비동기적으로 입력에 대해 체인을 호출한다.
* abatch : 비동기적으로 입력 목록에 대해 일괄 처리를 수행한다.
* astream_log : 최종 응답 및 발생하는 중간단계를 스트리밍한다.

In [1]:
from dotenv import load_dotenv

load_dotenv(dotenv_path="../.env")

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.1
)

In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate.from_template("{lecture}에 대해 3줄 요약해줘.")

chain = prompt | llm | StrOutputParser()

### invoke

In [4]:
print(chain.invoke({"lecture": "리액트"}))

리액트(React)는 사용자 인터페이스를 구축하기 위한 JavaScript 라이브러리로, 컴포넌트 기반 아키텍처를 사용합니다. 가상 DOM을 통해 효율적인 렌더링을 지원하며, 상태 관리와 생명주기 메서드를 통해 동적인 웹 애플리케이션 개발을 용이하게 합니다. 또한, 리액트 생태계에는 다양한 도구와 라이브러리가 있어 개발자들이 생산성을 높일 수 있도록 돕습니다.


### stream [실시간 출력]
* 데이터 스트림을 생성하고, 스트림을 반복하여 각 데이터의 내용을 즉시 출력한다.
* end="" 인자를 사용하기 때문에 줄바꿈을 하지 않을 수 있다.
* flush=True 인자를 사용하여 즉시 출력이 가능하다.

In [5]:
for token in chain.stream({"lecture": "Python"}):
    print(token, end="", flush=True)

Python은 간결하고 읽기 쉬운 문법을 가진 고급 프로그래밍 언어로, 다양한 분야에서 널리 사용됩니다. 객체 지향, 절차 지향, 함수형 프로그래밍을 지원하며, 방대한 라이브러리와 프레임워크를 통해 데이터 분석, 웹 개발, 인공지능 등 다양한 응용 프로그램을 개발할 수 있습니다. 또한, 플랫폼 독립적이어서 Windows, macOS, Linux 등 여러 운영 체제에서 실행 가능합니다.

### batch [단위 실행]
* 여러 개의 딕셔너리를 포함하는 리스트를 인자로 받아 각각의 입력에 대한 체인을 생성한다.
* max_concurrency : 최대 병렬 처리 수를 지정할 수 있다.

In [10]:
print(chain.batch([
    {"lecture": "파이썬"},
    {"lecture": "자바"},
    {"lecture": "자바스크립트"},
    {"lecture": "SQL"}
]
,config={"max_concurency": 3}))

['파이썬은 간결하고 읽기 쉬운 문법을 가진 고급 프로그래밍 언어로, 다양한 분야에서 널리 사용됩니다. 데이터 분석, 웹 개발, 인공지능 등 여러 용도로 활용할 수 있으며, 방대한 라이브러리와 커뮤니티 지원이 특징입니다. 또한, 플랫폼 독립적이어서 다양한 운영체제에서 실행할 수 있습니다.', '자바는 객체 지향 프로그래밍 언어로, 플랫폼 독립성을 제공하는 JVM(Java Virtual Machine)에서 실행됩니다. 강력한 메모리 관리와 풍부한 라이브러리를 갖추고 있어 다양한 애플리케이션 개발에 적합합니다. 또한, 보안성과 멀티스레딩 기능을 지원하여 대규모 시스템에서도 효율적으로 사용할 수 있습니다.', '자바스크립트는 웹 브라우저에서 동작하는 프로그래밍 언어로, 주로 웹 페이지의 인터랙티브한 요소를 구현하는 데 사용됩니다. 클라이언트 사이드 스크립팅뿐만 아니라 Node.js와 같은 환경을 통해 서버 사이드 프로그래밍에도 활용됩니다. 다양한 라이브러리와 프레임워크(예: React, Angular, Vue.js)를 통해 개발자들이 효율적으로 웹 애플리케이션을 구축할 수 있도록 지원합니다.', 'SQL(Structured Query Language)은 데이터베이스 관리 시스템에서 데이터를 정의, 조작 및 쿼리하기 위한 표준 프로그래밍 언어입니다. 주로 관계형 데이터베이스에서 사용되며, 데이터 삽입, 업데이트, 삭제 및 검색을 위한 다양한 명령어를 제공합니다. SQL은 데이터베이스의 구조를 정의하는 DDL(Data Definition Language), 데이터 조작을 위한 DML(Data Manipulation Language), 데이터 제어를 위한 DCL(Data Control Language)로 구분됩니다.']


---
### Parallel(병렬성)
* LCEL을 사용하여 체인을 구성할 때 여러 체인을 동시에 실행할 수 있다.

In [11]:
chain1 = (
    PromptTemplate.from_template("{country}의 수도는 어디야?")
    | llm
    | StrOutputParser()
)

chain2 = (
    PromptTemplate.from_template("{country}의 면적은 얼마야?")
    | llm
    | StrOutputParser()
)

In [12]:
from langchain_core.runnables import RunnableParallel

# 병렬 실행 체인
combined = RunnableParallel(capital=chain1, area=chain2)

In [13]:
chain1.invoke({"country": "보스니아 헤르체고비나"})

'보스니아 헤르체고비나의 수도는 사라예보(Sarajevo)입니다.'

In [14]:
chain2.invoke({"country": "알제리"})

'알제리의 면적은 약 2,381,741 평방킬로미터로, 아프리카 대륙에서 가장 큰 나라 중 하나입니다.'

In [15]:
result = combined.invoke({"country": "가이아나"})

print(result)

{'capital': '가이아나의 수도는 조지타운(Georgetown)입니다.', 'area': '가이아나의 면적은 약 214,970 평방킬로미터입니다. 이는 남아메리카에 위치한 국가로, 북쪽은 대서양과 접하고 있습니다.'}


In [16]:
print(result["capital"])
print(result["area"])

가이아나의 수도는 조지타운(Georgetown)입니다.
가이아나의 면적은 약 214,970 평방킬로미터입니다. 이는 남아메리카에 위치한 국가로, 북쪽은 대서양과 접하고 있습니다.


### RunnablePassThrough
* 입력을 그대로 전달하는 역할
* 단독으로 호출될 경우 단순히 입력을 받아 그대로 전달한다.

In [17]:
llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.0
)

prompt = PromptTemplate.from_template("{num}의 약수를 알려줘")

chain = prompt | llm | StrOutputParser()

In [18]:
# invoke()를 통해 실행할 때에는 입력이 딕셔너리 형태여야 하지만 1개의 변수만 템플릿에 작성이 되었다면 값만 전달해도 된다.
chain.invoke(10)

'10의 약수는 1, 2, 5, 10입니다.'

In [20]:
from langchain_core.runnables import RunnablePassthrough

passthrough_chain = {"num": RunnablePassthrough()} | prompt | llm | StrOutputParser()


passthrough_chain.invoke({"num": 10})

'10의 약수는 1, 2, 5, 10입니다.'

**RunnablePassthrough.assign()**
* 입력값으로 들어온 값의  key/value쌍을 새롭게 할당된 key/value쌍과 합쳐준다.

In [21]:
RunnablePassthrough.assign(new_date = lambda x: x["num"] * 3).invoke({"num": 1})

{'num': 1, 'new_date': 3}

### RunnableLambda
* 입력값에 대한 추가 처리를 수행할 수 있다.
* 람다 함수를 사용하여 입력값에 대한 추가 처리를 진행한다.
* 날씨, 시간 등등

In [22]:
from datetime import datetime

def get_today():
    return datetime.now().strftime("%b-%d")

get_today()

'Oct-28'